<a href="https://colab.research.google.com/github/TS0713/misc_sentiment_analysis/blob/main/Audio_Diarization_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-language
!pip install --upgrade google-cloud-speech
!pip install pytube
!pip install ffmpeg-python
!pip install pydub
!pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 KB 4.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-language
    Found existing installation: google-cloud-language 2.6.1
    Uninstalling google-cloud-language-2.6.1:
      Successfully uninstalled google-cloud-language-2.6.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.7/271.7 KB 7.2 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 2.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


## Loading all Necessary Libraries

In [ ]:
import json
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from google.cloud import language_v1
from pydub import AudioSegment
from pydub.silence import split_on_silence
from google.cloud import speech
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech

#json_file = "/content/sentimentanalysis-379314-a4a6d6259d79.json"
json_file = "/content/sentimentanalysis-379314-0c54bdecee97.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=json_file

In [ ]:

audio_file = "/content/drive/MyDrive/Datasets/phone_call.wav"


<_io.BufferedRandom name='audio_50_1.wav'>

<_io.BufferedRandom name='audio_50_2.wav'>

<_io.BufferedRandom name='audio_50_3.wav'>

<_io.BufferedRandom name='audio_50_4.wav'>

40.0


In [ ]:

#speech_file = "/content/Call Center Services.mp3"
speech_file = "/content/drive/MyDrive/Datasets/phone_call.wav"
#speech_file = "/content/audio50.wav"
client = speech.SpeechClient()

with open(speech_file, "rb") as audio_file:
    content = audio_file.read()


## Function to upload File/ Audio File to Google Cloud Storage Bucket

In [ ]:
def upload_file_GCS(colab_file_path,bucket_name,destination_name):

  
  storage_client = storage.Client()
  #bucket_name = "tsp-sample"
  bucket = storage_client.bucket(bucket_name)
  #blob = bucket.blob("call_recordings.wav")
  blob = bucket.blob(destination_name)
  generation_match_precondition = 0
  #source_file_name = "/content/phone_call.wav"
  source_file_name = colab_file_path
  try:
    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)
    print ("Successfully Uploaded  "+colab_file_path +"  to GCS Bucket - "+ bucket_name)
  except Exception as e:
    print ("Failed to Uploaded  "+colab_file_path +"  to GCS Bucket - "+ bucket_name)
    print (e)


## Function to get Speakers Details with text based on Input Audio File using Google Speech Client




In [ ]:
def create_transcript(enter_audio_file,uri=False,model_name=None):
  
  #speech_file = enter_audio_file 
  client = speech.SpeechClient()
  if uri:
    audio = speech.RecognitionAudio(uri=enter_audio_file)
  else:
    with open(enter_audio_file, "rb") as audio_file:
      content = audio_file.read()
      audio = speech.RecognitionAudio(content=content)
    
  
  diarization_config = speech.SpeakerDiarizationConfig(
    enable_speaker_diarization=True,
    min_speaker_count=2,
    max_speaker_count=10
    )
  if model_name is None:
    model_name = "latest_long"
    
  config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="en-US",
    diarization_config=diarization_config,
    model=model_name #"latest_long"    
    )
  try:
    if uri:
      operation = client.long_running_recognize(config=config, audio=audio)
      response = operation.result(timeout=90)

    else:
      response = client.recognize(config=config, audio=audio)

  except Exception as e:
    print ("==============================")
    print (e)
    print ("==============================")
  
  result = response.results[-1]
  
  words_info = result.alternatives[0].words
  text_script = [(word_info.word, "speaker_tag: "+str(word_info.speaker_tag)) for word_info in words_info]
  return text_script


## Uploading File to google cloud storage bucket

In [ ]:
upload_file_GCS(
    colab_file_path = "/content/phone_call.wav",
    bucket_name = "tsp-sample",
    destination_name = "new_audio.wav"
                )

Successfully Uploaded  /content/phone_call.wav  to GCS Bucket - tsp-sample


## Getting Speakers and Text Details based on Audio Input

In [ ]:
#inp_path = "/content/audio_50_1.wav"
inp_path = "gs://tsp-sample/new_phone_call.wav"
inp_path = "gs://tsp-sample/call_recordings.wav"
inp_path = "gs://tsp-sample/new_audio.wav"
audio_recording = create_transcript(enter_audio_file=inp_path,uri=True)
recordings = [i[0] for i in audio_recording]
print (" ".join(recordings))

hello hi Bob this is so how are you doing a pretty good Joe been a while since you called yeah things have been bit busy off late but with the dollar moving around the way it is doing now I thought I would check up on you you did mention that you have an important to cover for next month let's see it was about a million USD wasn't it I would say that this is a good time to book a forward cover for that Bob the pound has already fallen into one point two and it's likely to be further Joe I don't know why I'm even talking to you I'm sorry what was that I do all my banking with your bank so your Commercial Banking team has all my data why do you have to come and ask me if I have an import coming up and why aren't you aware of the exact amount of the invoice well how to manage so many customers you see so all the talk about being a trusted Financial partner for small and medium business is only talk is it lukier Joe I am a small business under million USD is a lot of money to me to so a fi

In [ ]:
audio_recording

[('hello', 'speaker_tag: 2'),
 ('hi', 'speaker_tag: 2'),
 ('Bob', 'speaker_tag: 2'),
 ('this', 'speaker_tag: 2'),
 ('is', 'speaker_tag: 2'),
 ('so', 'speaker_tag: 2'),
 ('how', 'speaker_tag: 2'),
 ('are', 'speaker_tag: 2'),
 ('you', 'speaker_tag: 2'),
 ('doing', 'speaker_tag: 2'),
 ('a', 'speaker_tag: 2'),
 ('pretty', 'speaker_tag: 2'),
 ('good', 'speaker_tag: 2'),
 ('Joe', 'speaker_tag: 2'),
 ('been', 'speaker_tag: 2'),
 ('a', 'speaker_tag: 2'),
 ('while', 'speaker_tag: 2'),
 ('since', 'speaker_tag: 2'),
 ('you', 'speaker_tag: 2'),
 ('called', 'speaker_tag: 2'),
 ('yeah', 'speaker_tag: 2'),
 ('things', 'speaker_tag: 2'),
 ('have', 'speaker_tag: 2'),
 ('been', 'speaker_tag: 2'),
 ('bit', 'speaker_tag: 2'),
 ('busy', 'speaker_tag: 2'),
 ('off', 'speaker_tag: 2'),
 ('late', 'speaker_tag: 2'),
 ('but', 'speaker_tag: 2'),
 ('with', 'speaker_tag: 2'),
 ('the', 'speaker_tag: 2'),
 ('dollar', 'speaker_tag: 2'),
 ('moving', 'speaker_tag: 2'),
 ('around', 'speaker_tag: 2'),
 ('the', 'speaker_ta

In [ ]:
#audio_50_1_convo = create_transcript("/content/audio_50_1.wav")#,model_name="phone_call")
#audio_50_2_convo = create_transcript("/content/audio_50_2.wav")#,model_name="phone_call")
#audio_50_3_convo = create_transcript("/content/audio_50_3.wav")#,model_name="phone_call")
#audio_50_4_convo = create_transcript("/content/audio_50_4.wav")#,model_name="phone_call")